** FIRST STEP IS TO INSTALL DEPENDENCIES **

In [1]:
#Dependencies
#! pip install pandas-datareader
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import warnings

** Next step provides selected closing index values.  This step uses datareader.  **

In [2]:
#Choose the ticker symbols for analysis

tickers = ['^IXIC', '^DJI', '^GSPC', '^RUT', '^GDAXI', '^XAX']

# Define which online source one should use
data_source = 'yahoo'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2017-11-14'
end_date = '2017-12-15'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader(tickers, data_source, start_date, end_date)

# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
adj_close = panel_data.ix['Adj Close']

# Getting all weekdays between 01/01/2000 and 12/31/2016
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

# How do we align the existing prices in adj_close with our new set of dates?
# All we need to do is reindex adj_close using all_weekdays as the new index
adj_close = adj_close.reindex(all_weekdays)

# Reindexing will insert missing values (NaN) for the dates that were not present
# in the original set. To cope with this, we can fill the missing by replacing them
# with the latest available price for each instrument.
adj_close = adj_close.fillna(method='ffill')

/Users/jbui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


** The index for the adjusted closing table is currently the date.  Need to change this for later steps. **

In [3]:
adj_close = adj_close.reset_index()
adj_close
#adj_close

,index,^DJI,^GDAXI,^GSPC,^IXIC,^RUT,^XAX
0,2017-11-14,23409.470703,13033.480469,2578.870117,6737.870117,1471.260010,2586.260010
1,2017-11-15,23271.279297,12976.370117,2564.620117,6706.209961,1464.089966,2579.510010
2,2017-11-16,23458.359375,13047.219727,2585.639893,6793.290039,1486.880005,2593.939941
3,2017-11-17,23358.240234,12993.730469,2578.850098,6782.790039,1492.819946,2601.479980
4,2017-11-20,23430.330078,13058.660156,2582.139893,6790.709961,1503.400024,2577.320068
5,2017-11-21,23590.830078,13167.540039,2599.030029,6862.479980,1518.890015,2587.949951
6,2017-11-22,23526.179688,13015.040039,2597.080078,6867.359863,1516.760010,2600.139893
7,2017-11-23,23526.179688,13008.549805,2597.080078,6867.359863,1516.760010,2600.139893
8,2017-11-24,23557.990234,13059.839844,2602.419922,6889.160156,1519.160034,2604.129883
9,2017-11-27,23580.779297,13000.200195,2601.419922,6878.520020,1513.310059,2585.100098


** To put performance of the various indices on the same scale - calculate one day returns (day (t) / day (t-1) - 1) - No value for the first day with index values - set it to 0 - will eliminate that row when you combine with the twitter data.  Should make sure the first day of finance data is earlier than first date of twitter data. **


In [4]:
#Calculate one day returns

count = 0
daily_DJI = []
daily_GSPC = []
daily_IXIC = []
daily_RUT = []
#daily_BTC = []
daily_GDAXI = []
daily_XAX = []

for index, row in adj_close.iterrows():
   
        if count == 0:
            Dailydji = 0
            Dailygspc = 0
            Dailyixic = 0
            Dailyrut = 0
            #Dailybtc = 0
            Dailygdaxi = 0
            Dailyxax = 0
            count = count + 1
            daily_DJI.append(Dailydji)
            daily_GSPC.append(Dailygspc)
            daily_IXIC.append(Dailyixic)
            daily_RUT.append(Dailyrut)
            daily_GDAXI.append(Dailygdaxi)
            daily_XAX.append(Dailyxax)
            #daily_BTC.append(Dailybtc)
            #print (count)
        else:
        
            Dailydji = adj_close["^DJI"][count]/adj_close["^DJI"][count - 1] - 1
            Dailygspc = adj_close["^GSPC"][count]/adj_close["^GSPC"][count - 1] - 1
            Dailyixic = adj_close["^IXIC"][count]/adj_close["^IXIC"][count - 1] - 1
            Dailyrut = adj_close["^RUT"][count]/adj_close["^RUT"][count - 1] - 1
            Dailygdaxi = adj_close["^GDAXI"][count]/adj_close["^GDAXI"][count - 1] - 1
            Dailyxax = adj_close["^XAX"][count]/adj_close["^XAX"][count - 1] - 1
            #Dailybtc = adj_close["BTC-USD"][count]/adj_close["BTC-USD"][count-1] -1
            count = count + 1
            daily_DJI.append(Dailydji)
            daily_GSPC.append(Dailygspc)
            daily_IXIC.append(Dailyixic)
            daily_RUT.append(Dailyrut)
            #daily_BTC.append(Dailybtc)
            daily_GDAXI.append(Dailygdaxi)
            daily_XAX.append(Dailyxax)
            #adj_close.loc[index,'Daily'] = adj_close["^DJI"][count]/adj_close["^DJI"][count - 1] - 1
            #print (count)
            
        #print("index is" + str(index))
        #print("row is" + str(row))

daily_DJI = pd.DataFrame({"Daily DJI":daily_DJI})
daily_GSPC = pd.DataFrame({"Daily GSPC":daily_GSPC})
daily_IXIC = pd.DataFrame({"Daily IXIC":daily_IXIC})
daily_RUT = pd.DataFrame({"Daily RUT":daily_RUT})
daily_GDAXI = pd.DataFrame({"Daily GDAXI":daily_GDAXI})
daily_XAX = pd.DataFrame({"Daily XAX":daily_XAX})
#daily_BTC = pd.DataFrame({"Daily BTC":daily_BTC})
merged = adj_close.join(daily_RUT,how = "inner")
merged = merged.join(daily_DJI,how = "inner")
merged = merged.join(daily_GSPC,how = "inner")
merged = merged.join(daily_IXIC,how = "inner")
merged = merged.join(daily_GDAXI,how = "inner")
merged = merged.join(daily_XAX,how = "inner")
merged.rename(columns={"index":"Date"},inplace=True)

In [6]:
merged = merged.sort_values(by='Date').drop(merged.index[0:13])

In [7]:
merged = merged.reset_index(drop=True)
merged

,Date,^DJI,^GDAXI,^GSPC,^IXIC,^RUT,^XAX,Daily RUT,Daily DJI,Daily GSPC,Daily IXIC,Daily GDAXI,Daily XAX
0,2017-12-01,24231.589844,12861.490234,2642.219971,6847.589844,1537.020020,2614.100098,-0.004611,-0.001679,-0.002025,-0.003838,-0.012476,0.002593
1,2017-12-04,24290.050781,13058.549805,2639.439941,6775.370117,1532.410034,2608.870117,-0.002999,0.002413,-0.001052,-0.010547,0.015322,-0.002001
2,2017-12-05,24180.640625,13048.540039,2629.570068,6762.209961,1516.760010,2590.080078,-0.010213,-0.004504,-0.003739,-0.001942,-0.000767,-0.007202
3,2017-12-06,24140.910156,12998.849609,2629.270020,6776.379883,1508.880005,2562.919922,-0.005195,-0.001643,-0.000114,0.002095,-0.003808,-0.010486
4,2017-12-07,24211.480469,13045.150391,2636.979980,6812.839844,1520.469971,2563.050049,0.007681,0.002923,0.002932,0.005380,0.003562,0.000051
5,2017-12-08,24329.160156,13153.700195,2651.500000,6840.080078,1521.719971,2566.820068,0.000822,0.004860,0.005506,0.003998,0.008321,0.001471
6,2017-12-11,24386.029297,13123.650391,2659.989990,6875.080078,1519.839966,2568.790039,-0.001235,0.002337,0.003202,0.005117,-0.002285,0.000767
7,2017-12-12,24504.800781,13183.530273,2664.110107,6862.319824,1516.119995,2562.399902,-0.002448,0.004870,0.001549,-0.001856,0.004563,-0.002488
8,2017-12-13,24585.429688,13125.639648,2662.850098,6875.799805,1524.449951,2560.699951,0.005494,0.003290,-0.000473,0.001964,-0.004391,-0.000663
9,2017-12-14,24508.660156,13068.080078,2652.010010,6856.529785,1506.949951,2559.790039,-0.011480,-0.003123,-0.004071,-0.002803,-0.004385,-0.000355


** Next - import the target user dataset that Ethan provided.  Group by date and user.  Want to line up twitter sentiment and index performance by date.  Need to make sure they are lined up if you are using a generic index.  Use the default date format for datetime in order to standardize for merging tables later.**

In [8]:
target_users = pd.read_csv("TargetUsers.csv")

target_users = target_users.groupby(["Date","UserName"]).mean()
target_users = target_users.reset_index()

target_users_CNN_Money = target_users.loc[target_users['UserName'] == 'CNNMoney']
target_users_WSJ_Markets = target_users.loc[target_users['UserName'] == 'WSJ Markets']
target_users_CNBC = target_users.loc[target_users['UserName'] == 'CNBC']
target_users_Financial_Times = target_users.loc[target_users['UserName'] == 'Financial Times']
target_users_Bloomberg_Markets = target_users.loc[target_users['UserName'] == 'Bloomberg Markets']
target_users_Justin_Wolfers = target_users.loc[target_users['UserName'] == 'Justin Wolfers']
target_users_Paul_Krugman = target_users.loc[target_users['UserName'] == 'Paul Krugman']
target_users_Jim_Cramer = target_users.loc[target_users['UserName'] == 'Jim Cramer']
target_users_Motley_Fool = target_users.loc[target_users['UserName'] == 'The Motley Fool']

from datetime import datetime
target_users_CNN_Money['Date'] = pd.to_datetime(target_users_CNN_Money['Date'])
target_users_WSJ_Markets['Date'] = pd.to_datetime(target_users_WSJ_Markets['Date'])
target_users_CNBC['Date'] = pd.to_datetime(target_users_CNBC['Date'])
target_users_Financial_Times['Date'] = pd.to_datetime(target_users_Financial_Times['Date'])
target_users_Bloomberg_Markets['Date'] = pd.to_datetime(target_users_Bloomberg_Markets['Date'])
target_users_Justin_Wolfers['Date'] = pd.to_datetime(target_users_Justin_Wolfers['Date'])
target_users_Paul_Krugman['Date'] = pd.to_datetime(target_users_Paul_Krugman['Date'])
target_users_Jim_Cramer['Date'] = pd.to_datetime(target_users_Jim_Cramer['Date'])
target_users_Motley_Fool['Date'] = pd.to_datetime(target_users_Motley_Fool['Date'])


/Users/jbui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jbui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jbui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

In [10]:
target_users_CNN_Money

,Date,UserName,Compound
2,2017-12-01,CNNMoney,0.159346
11,2017-12-02,CNNMoney,0.102483
20,2017-12-03,CNNMoney,0.015344
29,2017-12-04,CNNMoney,0.193639
38,2017-12-05,CNNMoney,0.032921
47,2017-12-06,CNNMoney,0.062312
56,2017-12-07,CNNMoney,0.074863
65,2017-12-08,CNNMoney,0.005354
73,2017-12-09,CNNMoney,0.177970
82,2017-12-10,CNNMoney,0.069915


** Now combine the twitter data with the market data into one final table called combined_table.  **

In [22]:
combined_table = pd.merge(target_users_CNN_Money, merged, on="Date", how="inner")
combined_table = combined_table.drop(['UserName','^DJI',"^GDAXI","^GSPC","^IXIC","^RUT","^XAX"], axis=1)
combined_table.rename(columns={"Compound":"CNN Sentiment"},inplace=True)

combined_table = pd.merge(target_users_WSJ_Markets, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"WSJ Sentiment"},inplace=True)

combined_table = pd.merge(target_users_CNBC, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"CNBC Sentiment"},inplace=True)

combined_table = pd.merge(target_users_Financial_Times, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"Fin Times Sentiment"},inplace=True)

combined_table = pd.merge(target_users_Bloomberg_Markets, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"Bloomberg Sentiment"},inplace=True)

combined_table = pd.merge(target_users_Justin_Wolfers, combined_table, on="Date", how="outer")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"Justin Wolfers Sentiment"},inplace=True)

combined_table = pd.merge(target_users_Paul_Krugman, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"Paul Krugman Sentiment"},inplace=True)

combined_table = pd.merge(target_users_Jim_Cramer, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"Jim Cramer Sentiment"},inplace=True)

combined_table = pd.merge(target_users_Motley_Fool, combined_table, on="Date", how="inner")
combined_table = combined_table.drop(['UserName'], axis=1)
combined_table.rename(columns={"Compound":"Motley Fool Sentiment"},inplace=True)

combined_table = combined_table.sort_values(by='Date')
combined_table = combined_table.reset_index(drop=True)
combined_table = combined_table.drop(combined_table.index[[1,2,8,9]])
combined_table = combined_table.reset_index(drop=True)
combined_table = combined_table.fillna(0)

In [23]:
combined_table2 = combined_table.iloc[0:10,:]
combined_table2 = combined_table2.drop(combined_table2.index[0]).reset_index(drop=True)
combined_table2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 16 columns):
Date                        9 non-null datetime64[ns]
Motley Fool Sentiment       9 non-null float64
Jim Cramer Sentiment        9 non-null float64
Paul Krugman Sentiment      9 non-null float64
Justin Wolfers Sentiment    9 non-null float64
Bloomberg Sentiment         9 non-null float64
Fin Times Sentiment         9 non-null float64
CNBC Sentiment              9 non-null float64
WSJ Sentiment               9 non-null float64
CNN Sentiment               9 non-null float64
Daily RUT                   9 non-null float64
Daily DJI                   9 non-null float64
Daily GSPC                  9 non-null float64
Daily IXIC                  9 non-null float64
Daily GDAXI                 9 non-null float64
Daily XAX                   9 non-null float64
dtypes: datetime64[ns](1), float64(15)
memory usage: 1.2 KB


In [24]:
combined_table1 = combined_table.iloc[0:9,0:10]
combined_table1 = combined_table1.reset_index(drop=True)
combined_table1 = combined_table1.rename(columns={"Motley Fool Sentiment":"prior motley","Jim Cramer Sentiment":"prior jim","Paul Krugman Sentiment":"prior paul","Justin Wolfers Sentiment":"prior justin","Bloomberg Sentiment":"prior bloomberg","Fin Times Sentiment":"prior ft","CNBC Sentiment":"prior cnbc","WSJ Sentiment":"prior wsj","CNN Sentiment":"prior cnn"})
combined_table1 = combined_table1.drop(['Date'], axis=1)
combined_table1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 9 columns):
prior motley       9 non-null float64
prior jim          9 non-null float64
prior paul         9 non-null float64
prior justin       9 non-null float64
prior bloomberg    9 non-null float64
prior ft           9 non-null float64
prior cnbc         9 non-null float64
prior wsj          9 non-null float64
prior cnn          9 non-null float64
dtypes: float64(9)
memory usage: 728.0 bytes


In [25]:
final_table = pd.concat([combined_table2, combined_table1], axis=1, join_axes=[combined_table2.index])
final_table.head()

,Date,Motley Fool Sentiment,Jim Cramer Sentiment,Paul Krugman Sentiment,Justin Wolfers Sentiment,Bloomberg Sentiment,Fin Times Sentiment,CNBC Sentiment,WSJ Sentiment,CNN Sentiment,...,Daily XAX,prior motley,prior jim,prior paul,prior justin,prior bloomberg,prior ft,prior cnbc,prior wsj,prior cnn
0,2017-12-04,0.545300,0.331550,0.128700,-0.515550,0.163638,-0.094769,0.045210,-0.091178,0.193639,...,-0.002001,0.543200,0.091480,0.002890,0.167243,-0.023289,0.040479,0.101311,0.169091,0.159346
1,2017-12-05,0.418414,0.153906,-0.111331,-0.888500,0.050363,-0.007009,0.095859,-0.073602,0.032921,...,-0.007202,0.545300,0.331550,0.128700,-0.515550,0.163638,-0.094769,0.045210,-0.091178,0.193639
2,2017-12-06,0.391400,0.095669,-0.445800,0.199525,-0.037715,-0.075401,0.099526,0.139922,0.062312,...,-0.010486,0.418414,0.153906,-0.111331,-0.888500,0.050363,-0.007009,0.095859,-0.073602,0.032921
3,2017-12-07,0.235750,0.226179,0.078050,0.096714,-0.031520,-0.003583,0.027770,0.356296,0.074863,...,0.000051,0.391400,0.095669,-0.445800,0.199525,-0.037715,-0.075401,0.099526,0.139922,0.062312
4,2017-12-08,0.028833,-0.052367,-0.094853,-0.106050,0.112100,0.040034,0.173727,0.169653,0.005354,...,0.001471,0.235750,0.226179,0.078050,0.096714,-0.031520,-0.003583,0.027770,0.356296,0.074863


In [26]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 25 columns):
Date                        9 non-null datetime64[ns]
Motley Fool Sentiment       9 non-null float64
Jim Cramer Sentiment        9 non-null float64
Paul Krugman Sentiment      9 non-null float64
Justin Wolfers Sentiment    9 non-null float64
Bloomberg Sentiment         9 non-null float64
Fin Times Sentiment         9 non-null float64
CNBC Sentiment              9 non-null float64
WSJ Sentiment               9 non-null float64
CNN Sentiment               9 non-null float64
Daily RUT                   9 non-null float64
Daily DJI                   9 non-null float64
Daily GSPC                  9 non-null float64
Daily IXIC                  9 non-null float64
Daily GDAXI                 9 non-null float64
Daily XAX                   9 non-null float64
prior motley                9 non-null float64
prior jim                   9 non-null float64
prior paul                  9 non-null float64

** Now for some factor generation.  Start by lining up market performance with prior day's tweet sentiment.  Want to see if you can use tweet sentiment to predict the market.  Use this for an investment strategy.  **

In [27]:
table = pd.read_csv("target_terms.csv")
#final_table.to_csv('combined_table2.csv')
#combined_table = pd.read_csv("combined_table2.csv")
table = table.drop(['Unnamed: 0','Date'], axis=1)
table

,target dow,target nasdaq,target nyse,target dow prior,target nasdaq prior,target nyse prior
0,0.023830,0.188898,0.349148,0.112647,0.226000,0.216636
1,-0.014471,0.153072,0.290614,0.023830,0.188898,0.349148
2,-0.068465,0.257540,0.246785,-0.014471,0.153072,0.290614
3,-0.013719,0.312886,0.259690,-0.068465,0.257540,0.246785
4,0.197037,0.352862,0.130623,-0.013719,0.312886,0.259690
5,0.155555,0.332114,0.261354,0.197037,0.352862,0.130623
6,0.177248,0.260784,0.260063,0.155555,0.332114,0.261354
7,0.124631,0.312702,0.276952,0.177248,0.260784,0.260063
8,0.095950,0.315387,0.311375,0.124631,0.312702,0.276952


In [28]:
final_table = pd.concat([final_table, table], axis=1, join_axes=[final_table.index])
final_table.to_csv('combined_table.csv')
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 31 columns):
Date                        9 non-null datetime64[ns]
Motley Fool Sentiment       9 non-null float64
Jim Cramer Sentiment        9 non-null float64
Paul Krugman Sentiment      9 non-null float64
Justin Wolfers Sentiment    9 non-null float64
Bloomberg Sentiment         9 non-null float64
Fin Times Sentiment         9 non-null float64
CNBC Sentiment              9 non-null float64
WSJ Sentiment               9 non-null float64
CNN Sentiment               9 non-null float64
Daily RUT                   9 non-null float64
Daily DJI                   9 non-null float64
Daily GSPC                  9 non-null float64
Daily IXIC                  9 non-null float64
Daily GDAXI                 9 non-null float64
Daily XAX                   9 non-null float64
prior motley                9 non-null float64
prior jim                   9 non-null float64
prior paul                  9 non-null float64

In [29]:
final_table

,Date,Motley Fool Sentiment,Jim Cramer Sentiment,Paul Krugman Sentiment,Justin Wolfers Sentiment,Bloomberg Sentiment,Fin Times Sentiment,CNBC Sentiment,WSJ Sentiment,CNN Sentiment,...,prior ft,prior cnbc,prior wsj,prior cnn,target dow,target nasdaq,target nyse,target dow prior,target nasdaq prior,target nyse prior
0,2017-12-04,0.545300,0.331550,0.128700,-0.515550,0.163638,-0.094769,0.045210,-0.091178,0.193639,...,0.040479,0.101311,0.169091,0.159346,0.023830,0.188898,0.349148,0.112647,0.226000,0.216636
1,2017-12-05,0.418414,0.153906,-0.111331,-0.888500,0.050363,-0.007009,0.095859,-0.073602,0.032921,...,-0.094769,0.045210,-0.091178,0.193639,-0.014471,0.153072,0.290614,0.023830,0.188898,0.349148
2,2017-12-06,0.391400,0.095669,-0.445800,0.199525,-0.037715,-0.075401,0.099526,0.139922,0.062312,...,-0.007009,0.095859,-0.073602,0.032921,-0.068465,0.257540,0.246785,-0.014471,0.153072,0.290614
3,2017-12-07,0.235750,0.226179,0.078050,0.096714,-0.031520,-0.003583,0.027770,0.356296,0.074863,...,-0.075401,0.099526,0.139922,0.062312,-0.013719,0.312886,0.259690,-0.068465,0.257540,0.246785
4,2017-12-08,0.028833,-0.052367,-0.094853,-0.106050,0.112100,0.040034,0.173727,0.169653,0.005354,...,-0.003583,0.027770,0.356296,0.074863,0.197037,0.352862,0.130623,-0.013719,0.312886,0.259690
5,2017-12-11,0.164386,0.129338,-0.087900,0.000000,0.120875,-0.067214,0.085089,-0.107352,-0.070345,...,0.040034,0.173727,0.169653,0.005354,0.155555,0.332114,0.261354,0.197037,0.352862,0.130623
6,2017-12-12,0.515125,0.113133,-0.057460,0.393175,0.053395,0.055600,0.214850,0.041404,0.069605,...,-0.067214,0.085089,-0.107352,-0.070345,0.177248,0.260784,0.260063,0.155555,0.332114,0.261354
7,2017-12-13,0.019938,0.101578,0.306550,0.452590,-0.046800,0.057811,0.112263,0.137368,0.041081,...,0.055600,0.214850,0.041404,0.069605,0.124631,0.312702,0.276952,0.177248,0.260784,0.260063
8,2017-12-14,0.181017,0.311612,-0.056967,0.562050,0.125333,-0.007283,0.137195,0.084000,0.184064,...,0.057811,0.112263,0.137368,0.041081,0.095950,0.315387,0.311375,0.124631,0.312702,0.276952


** Now calculate the correlation matrix - and export it to "correlation.csv". **

In [30]:
corr = final_table.corr()
corr.to_csv('correlation_table.csv')
corr

,Motley Fool Sentiment,Jim Cramer Sentiment,Paul Krugman Sentiment,Justin Wolfers Sentiment,Bloomberg Sentiment,Fin Times Sentiment,CNBC Sentiment,WSJ Sentiment,CNN Sentiment,Daily RUT,...,prior ft,prior cnbc,prior wsj,prior cnn,target dow,target nasdaq,target nyse,target dow prior,target nasdaq prior,target nyse prior
Motley Fool Sentiment,1.000000,0.429243,-0.272771,-0.395300,0.149805,-0.420871,-0.090491,-0.410862,0.422686,-0.405592,...,-0.419137,-0.354473,-0.572958,0.105836,-0.453362,-0.809421,0.523066,-0.033077,-0.401449,0.197390
Jim Cramer Sentiment,0.429243,1.000000,0.279270,-0.057902,0.284277,-0.469097,-0.528226,-0.199645,0.756794,-0.289720,...,0.194024,0.180006,-0.078575,0.261420,-0.414414,-0.374993,0.888781,0.174123,-0.124683,-0.079654
Paul Krugman Sentiment,-0.272771,0.279270,1.000000,0.067810,0.025224,0.373182,-0.198348,0.060980,0.188840,0.516927,...,0.258168,0.481563,0.261628,0.203163,0.316465,0.101849,0.306885,0.387080,0.297058,-0.223477
Justin Wolfers Sentiment,-0.395300,-0.057902,0.067810,1.000000,-0.306379,0.377142,0.389643,0.472689,0.085536,0.237236,...,0.386728,0.486312,-0.012651,-0.793573,0.358267,0.698456,-0.117635,0.298701,0.432060,-0.183644
Bloomberg Sentiment,0.149805,0.284277,0.025224,-0.306379,1.000000,-0.313268,0.123247,-0.626539,0.228187,-0.448294,...,0.342284,-0.252760,0.505097,0.152029,0.397846,-0.074984,0.139557,0.339657,0.447489,-0.375770
Fin Times Sentiment,-0.420871,-0.469097,0.373182,0.377142,-0.313268,1.000000,0.643569,0.413261,-0.185279,0.300695,...,-0.247151,-0.034791,-0.052111,-0.286894,0.557606,0.345573,-0.432055,0.057491,0.386569,0.345940
CNBC Sentiment,-0.090491,-0.528226,-0.198348,0.389643,0.123247,0.643569,1.000000,-0.049561,-0.139962,-0.235481,...,-0.055970,-0.243571,-0.131042,-0.545256,0.675780,0.238807,-0.458945,0.271432,0.457256,0.221274
WSJ Sentiment,-0.410862,-0.199645,0.060980,0.472689,-0.626539,0.413261,-0.049561,1.000000,0.045141,0.576232,...,-0.220221,-0.069946,0.172253,-0.225884,-0.136569,0.518601,-0.435184,-0.614229,-0.063050,0.223576
CNN Sentiment,0.422686,0.756794,0.188840,0.085536,0.228187,-0.185279,-0.139962,0.045141,1.000000,-0.318320,...,0.193150,-0.134636,-0.037421,0.232315,-0.325450,-0.342934,0.603008,-0.032942,-0.246798,0.282161
Daily RUT,-0.405592,-0.289720,0.516927,0.237236,-0.448294,0.300695,-0.235481,0.576232,-0.318320,1.000000,...,-0.019599,0.361113,0.317173,-0.193109,0.187182,0.486205,-0.333359,-0.124284,0.184788,-0.393270
